<a href="https://colab.research.google.com/github/Lokeshrathi/PyTorch/blob/main/Pre_Processing_Text_Datasets_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext==0.10.0

import torch
from torchtext.legacy import data, datasets
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 5.1 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [4]:
seed = 966
torch.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'gpu') ## if prints CUDA, then we are using GPU else CPU
print(device)

cuda


In [6]:
## define Feilds

TEXT = data.Field(tokenize='spacy', lower=True)
LABEL = data.LabelField()

/usr/local/lib/python3.7/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [7]:
train,test = datasets.TREC.splits(TEXT, LABEL)
train, val = train.split(random_state=random.seed(seed))


downloading train_5500.label


train_5500.label: 100%|██████████| 336k/336k [00:00<00:00, 3.02MB/s]


downloading TREC_10.label


TREC_10.label: 100%|██████████| 23.4k/23.4k [00:00<00:00, 915kB/s]


In [10]:
## build Vocabulary

TEXT.build_vocab(train,min_freq=2)
LABEL.build_vocab(train)

In [11]:
print(LABEL.vocab.stoi)

defaultdict(None, {'ENTY': 0, 'HUM': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})


In [13]:
print("Length of vocab in Train Text",len(TEXT.vocab.stoi))

Length of vocab in Train Text 2643


In [15]:
train_iterator,valid_iterator,test_iterator = data.BucketIterator.splits(
    (train,test,val),
    batch_size=64,
    sort_key = lambda x:x.len(x.text),
    device = device
)